In this file, we will go through how we can train a model that will help us predict car prices using PyTorch. 


### What is PyTorch?

**PyTorch** is a library in Python which provides tools to build deep learning models. 
* Python is a very flexible language for programming and just like python, the PyTorch library provides flexible tools for deep learning. 
* If we are learning deep learning or looking to start with it, then the knowledge of PyTorch will help us a lot in creating our deep learning models.

In [4]:
# pip install jovian

In [3]:
import torch
import jovian
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split

<IPython.core.display.Javascript object>

In [5]:
dataframe_raw = pd.read_csv("car data.csv")
dataframe_raw.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


We can see how the data looks like. But before using it, we need to customize it, sort the arrows and remove the columns that don’t help the prediction.

Here we drop the car names, and to do this customization, we use the following function:

In [6]:
your_name = "Waqas Ali" # at least 5 characters

def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.Year = dataframe.Year * ord(rand_str[1])/100.
    # scale target
    dataframe.Selling_Price = dataframe.Selling_Price * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['Car_Name'], axis=1)
    return dataframe

In [7]:
dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.head()

,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
183,1952.61,0.3051,0.47,21000,Petrol,Individual,Manual,0
67,1949.70,10.4525,20.45,59000,Diesel,Dealer,Manual,0
149,1955.52,0.5763,0.94,24000,Petrol,Individual,Manual,0
92,1944.85,3.9663,13.70,75000,Petrol,Dealer,Manual,0
115,1954.55,1.2543,1.47,17500,Petrol,Individual,Manual,0


In the function above as we see it, needs a word to use as a random string to sort data randomly. I used my name as a random string. 

After that we can use the custom dataset. For simplicity we can create variables containing the number of rows, columns and variables containing the numeric, categorical or output columns:

In [8]:
input_cols = ["Year","Present_Price","Kms_Driven","Owner"]
categorical_cols = ["Fuel_Type","Seller_Type","Transmission"]
output_cols = ["Selling_Price"]

### Data Preparation

As stated at the beginning, we will be using PyTorch to predict car prices using machine learning. To use data for training, we need to convert it from dataframe to PyTorch Tensors.

In [9]:
# First step is to convert dataframe to NumPy arrays:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

In [11]:
inputs_array, targets_array = dataframe_to_arrays(dataframe)

In [12]:
inputs_array, targets_array

(array([[1.95261e+03, 4.70000e-01, 2.10000e+04, 0.00000e+00],
        [1.94970e+03, 2.04500e+01, 5.90000e+04, 0.00000e+00],
        [1.95552e+03, 9.40000e-01, 2.40000e+04, 0.00000e+00],
        ...,
        [1.95552e+03, 1.36000e+01, 1.09800e+04, 0.00000e+00],
        [1.95261e+03, 1.50000e+00, 1.50000e+04, 0.00000e+00],
        [1.95261e+03, 9.90000e+00, 5.42420e+04, 0.00000e+00]]),
 array([[ 0.3051],
        [10.4525],
        [ 0.5763],
        [ 3.9663],
        [ 1.2543],
        [19.21  ],
        [ 0.226 ],
        [ 5.085 ],
        [20.34  ],
        [ 3.6725],
        [ 7.9665],
        [ 6.893 ],
        [ 0.452 ],
        [ 1.4125],
        [ 0.678 ],
        [ 8.475 ],
        [ 1.2995],
        [ 8.362 ],
        [ 4.294 ],
        [ 9.6615],
        [37.29  ],
        [ 4.633 ],
        [16.6449],
        [ 2.5425],
        [ 9.8875],
        [ 1.6385],
        [ 0.4294],
        [16.1025],
        [ 5.537 ],
        [ 6.4975],
        [ 3.5595],
        [ 3.503 ],
     

The above function converts the input and output columns to NumPy arrays. Now having these arrays, we can convert them to PyTorch tensors, and use those tensors to create a variable dataset that contains them:

In [13]:
inputs = torch.Tensor(inputs_array)
targets = torch.Tensor(targets_array)

In [14]:
dataset = TensorDataset(inputs, targets)
train_ds, val_ds = random_split(dataset, [228, 57])

In [15]:
batch_size = 128

train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

In [16]:
# creating linear regressing model using PyTorch to predict car prices:

input_size = len(input_cols)
output_size = len(output_cols)

class CarsModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)
        
    def forward(self, xb):
        out = self.linear(xb)                          
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        loss = F.l1_loss(out, targets)                         
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.l1_loss(out, targets)                           
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

In [17]:
model = CarsModel()

list(model.parameters())

[Parameter containing:
 tensor([[-0.1670, -0.1796,  0.2362,  0.1503]], requires_grad=True),
 Parameter containing:
 tensor([-0.4753], requires_grad=True)]

In this above function, we used the `nn.Linear` function which will allow us to use linear regression. 

Now we can make predictions and calculatethe loss with the `F.l1_loss` function. We can also see the weight parameters and bias. 

In [18]:
# Eval algorithm
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [19]:
# Fitting algorithm
def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

In [20]:
# Check the initial value that val_loss have
result = evaluate(model, val_loader)
print(result)

{'val_loss': 7891.85205078125}


In [21]:
# Start with the Fitting
epochs = 90
lr = 1e-8
history1 = fit(epochs, lr, model, train_loader, val_loader)

C:\ProgramData\Anaconda3\lib\site-packages\torch\autograd\__init__.py:132: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  allow_unreachable=True)  # allow_unreachable flag


Epoch [20], val_loss: 7371.2520
Epoch [40], val_loss: 6849.9141
Epoch [60], val_loss: 6328.8125
Epoch [80], val_loss: 5807.2832
Epoch [90], val_loss: 5547.7808


As we can see, **evaluation** and **fit** functions are used to do training. We use optimization functions (in this case specifically SGD optimization) using **train loader** to calculate the loss & gradients and to optimize it afterwards & evaluate the result of each iteration to see the loss.

Finally, we need to test the model with specific data, to predict.

In [22]:
# Prediction Algorithm
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)                
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

In [23]:
# Testing the model with some samples
input, target = val_ds[0]
predict_single(input, target, model)

Input: tensor([1.9516e+03, 9.4000e+00, 3.6100e+04, 0.0000e+00])
Target: tensor([5.0850])
Prediction: tensor([5760.4648])


As we can see, the predictions are not very close to the expected target. With this we can test different results and see how good the model is:

In [24]:
input, target = val_ds[10]
predict_single(input, target, model)

Input: tensor([1.9497e+03, 7.9800e+00, 4.1442e+04, 0.0000e+00])
Target: tensor([2.9945])
Prediction: tensor([6663.0059])
